In [17]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.impute import SimpleImputer

In [2]:
boston = pd.read_csv("Boston.csv")
X = boston.drop('medv',axis=1)
y = boston['medv']

In [3]:
lr = LinearRegression()
result = cross_val_score(lr, X, y)
result.mean()

0.3532759243958806

# Elastic Net

In [4]:
elastic = ElasticNet()
result = cross_val_score(elastic, X, y)
result.mean()

0.472652519194106

- Hyper_parameter Optimization

In [5]:
alphas = np.linspace(0.0001, 10, 20)
l1 = np.linspace(0.0001, 1, 10)
scores = []
for a in alphas:
    for i in l1:
        elastic = ElasticNet(alpha=a, l1_ratio=i)
        result = cross_val_score(elastic, X, y)
        scores.append([a,i, result.mean()])
df_scores = pd.DataFrame(scores, columns=['alpha','l1_ratio','score'])
df_scores.sort_values('score', ascending=False, inplace=True)
best_a = df_scores['alpha'].iloc[0]
best_sc = df_scores['score'].iloc[0]
best_l1 = df_scores['l1_ratio'].iloc[0]
print("Best Alpha :     ", best_a)
print("Best L1 ratio :  ", best_l1)
print("Best Score :     ", best_sc)

Best Alpha :      0.5264105263157894
Best L1 ratio :   0.0001
Best Score :      0.4976889030997101


In [6]:
df_scores

,alpha,l1_ratio,score
10,0.526411,0.0001,0.497689
11,0.526411,0.1112,0.495480
12,0.526411,0.2223,0.492985
13,0.526411,0.3334,0.490010
20,1.052721,0.0001,0.487077
...,...,...,...
188,9.473689,0.8889,0.284385
179,8.947379,1.0000,0.283123
198,10.000000,0.8889,0.280179
189,9.473689,1.0000,0.278435


# GridSearch CV
- from `sklearn.model_selection` import `GridSearchCV`


In [7]:
elastic = ElasticNet()
params = {'alpha': np.linspace(0.0001,10,20), 'l1_ratio': np.linspace(0.0001,1,10)}    # 'alpha' & 'l1_ratio' are the parameters of elastic net algo
gridscv = GridSearchCV(elastic, param_grid=params)

In [8]:
elastic.get_params()    # To get all available parameters of elastic net

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 1000,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [9]:
gridscv.fit(X,y)

GridSearchCV(estimator=ElasticNet(),
             param_grid={'alpha': array([1.00000000e-04, 5.26410526e-01, 1.05272105e+00, 1.57903158e+00,
       2.10534211e+00, 2.63165263e+00, 3.15796316e+00, 3.68427368e+00,
       4.21058421e+00, 4.73689474e+00, 5.26320526e+00, 5.78951579e+00,
       6.31582632e+00, 6.84213684e+00, 7.36844737e+00, 7.89475789e+00,
       8.42106842e+00, 8.94737895e+00, 9.47368947e+00, 1.00000000e+01]),
                         'l1_ratio': array([1.000e-04, 1.112e-01, 2.223e-01, 3.334e-01, 4.445e-01, 5.556e-01,
       6.667e-01, 7.778e-01, 8.889e-01, 1.000e+00])})

In [10]:
print("Best params : ",gridscv.best_params_)
print("Best score  : ",gridscv.best_score_)

Best params :  {'alpha': 0.5264105263157894, 'l1_ratio': 0.0001}
Best score  :  0.4976889030997101


# Pipeline Using GridSearch CV

In [11]:
from sklearn.pipeline import Pipeline

In [12]:
chem = pd.read_csv('ChemicalProcess.csv')

In [13]:
X = chem.drop('Yield',axis=1)
y = chem['Yield']

- At hthe end of the pipline musy use estimator ( Object of Model Eg: lr, ct, elastic )

In [14]:
# With Mean Strategy

In [18]:
imp_mean = SimpleImputer(strategy='mean').set_output(transform='pandas')
lr = LinearRegression()
pipe = Pipeline([('IMP',imp_mean),('LR',lr)])
result_lr = cross_val_score(pipe, X, y)
result_lr.mean()

-94.57748595795053

In [19]:
# With Median strategy

In [20]:
imp_median = SimpleImputer(strategy='median').set_output(transform='pandas')
lr = LinearRegression()
pipe = Pipeline([('IMP',imp_median),('LR',lr)])
result_lr = cross_val_score(pipe, X, y)
result_lr.mean()

-86.99645845793464

In [21]:
pipe.get_params()        # To Get All Available Parameters in Dataset

{'memory': None,
 'steps': [('IMP', SimpleImputer(strategy='median')),
  ('LR', LinearRegression())],
 'verbose': False,
 'IMP': SimpleImputer(strategy='median'),
 'LR': LinearRegression(),
 'IMP__add_indicator': False,
 'IMP__copy': True,
 'IMP__fill_value': None,
 'IMP__keep_empty_features': False,
 'IMP__missing_values': nan,
 'IMP__strategy': 'median',
 'LR__copy_X': True,
 'LR__fit_intercept': True,
 'LR__n_jobs': None,
 'LR__positive': False}

In [22]:
# Below is the shortcut to automatically find best score between  mean & Median

In [23]:
imp = SimpleImputer()
pipe = Pipeline([('IMP',imp),('LR',lr)])
params = {'IMP__strategy':['mean','median']}
gridscv = GridSearchCV(pipe, param_grid=params)
gridscv.fit(X,y)
print("Best params : ",gridscv.best_params_)
print("Best score  : ",gridscv.best_score_)

Best params :  {'IMP__strategy': 'median'}
Best score  :  -86.99645845793464


# For Elastic Net

In [24]:
elastic.get_params()    # To get all available parameters of elastic net

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 1000,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [25]:

elastic = ElasticNet()
imp = SimpleImputer()
pipe = Pipeline([('IMP',imp),('el',elastic)])
pipe.get_params()

{'memory': None,
 'steps': [('IMP', SimpleImputer()), ('el', ElasticNet())],
 'verbose': False,
 'IMP': SimpleImputer(),
 'el': ElasticNet(),
 'IMP__add_indicator': False,
 'IMP__copy': True,
 'IMP__fill_value': None,
 'IMP__keep_empty_features': False,
 'IMP__missing_values': nan,
 'IMP__strategy': 'mean',
 'el__alpha': 1.0,
 'el__copy_X': True,
 'el__fit_intercept': True,
 'el__l1_ratio': 0.5,
 'el__max_iter': 1000,
 'el__positive': False,
 'el__precompute': False,
 'el__random_state': None,
 'el__selection': 'cyclic',
 'el__tol': 0.0001,
 'el__warm_start': False}

In [26]:
params = {'el__alpha': np.linspace(0.0001,10,20), 'el__l1_ratio': np.linspace(0.0001,1,10),'IMP__strategy':['mean','median']}    # 'alpha' & 'l1_ratio' are the parameters of elastic net algo
gridscv = GridSearchCV(pipe, param_grid=params)
gridscv.fit(X,y)

C:\Users\DAI.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.126e+01, tolerance: 4.218e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\DAI.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.350e+01, tolerance: 4.172e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\DAI.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

GridSearchCV(estimator=Pipeline(steps=[('IMP', SimpleImputer()),
                                       ('el', ElasticNet())]),
             param_grid={'IMP__strategy': ['mean', 'median'],
                         'el__alpha': array([1.00000000e-04, 5.26410526e-01, 1.05272105e+00, 1.57903158e+00,
       2.10534211e+00, 2.63165263e+00, 3.15796316e+00, 3.68427368e+00,
       4.21058421e+00, 4.73689474e+00, 5.26320526e+00, 5.78951579e+00,
       6.31582632e+00, 6.84213684e+00, 7.36844737e+00, 7.89475789e+00,
       8.42106842e+00, 8.94737895e+00, 9.47368947e+00, 1.00000000e+01]),
                         'el__l1_ratio': array([1.000e-04, 1.112e-01, 2.223e-01, 3.334e-01, 4.445e-01, 5.556e-01,
       6.667e-01, 7.778e-01, 8.889e-01, 1.000e+00])})

In [27]:
print("Best params : ",gridscv.best_params_)
print("Best score  : ",gridscv.best_score_)

Best params :  {'IMP__strategy': 'median', 'el__alpha': 10.0, 'el__l1_ratio': 1.0}
Best score  :  -1.55172620877767


# Grid View:

In [28]:
# how -1.55172620877767 this value is pick as best score let's check:

In [29]:
df_result = pd.DataFrame(gridscv.cv_results_)
df_result.shape

(400, 16)

In [30]:
# To export CSV and to cross check the output
df_result.to_csv('C:/Users/DAI.STUDENTSDC/Desktop/ /Machine Learning/cv_results.csv',index=False)

# Housing.csv

In [31]:
housing = pd.read_csv('Housing.csv')

In [32]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [33]:
X = housing.drop('price',axis=1)
y = housing['price']

In [34]:
ohe = OneHotEncoder(sparse_output=False,drop='first').set_output(transform='pandas')      
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude=object)),
                             (ohe,make_column_selector(dtype_include=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')
elastic = ElasticNet()
pipe = Pipeline([('CT',ct),('EL',elastic)])
pipe.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000023B2AF99160>),
                                   ('onehotencoder',
                                    OneHotEncoder(drop='first',
                                                  sparse_output=False),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000023B2CAACDA0>)],
                     verbose_feature_names_out=False)),
  ('EL', ElasticNet())],
 'verbose': False,
 'CT': ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000023B2AF99160>),
                                 ('onehotencoder',
                                  OneHotEncoder(drop='first',
                                                sp

In [35]:
params = {'EL__alpha': np.linspace(0.0001,10,20), 'EL__l1_ratio': np.linspace(0.0001,1,10)}    # 'alpha' & 'l1_ratio' are the parameters of elastic net algo
gridscv = GridSearchCV(pipe, param_grid=params)
gridscv.fit(X,y)

GridSearchCV(estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000023B2AF99160>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000023B2CAACDA0>)],
                                                          verbos...
       2.10534211e+00, 2.63165263e+00, 3.15796316e+00, 3.68427368e+00,
       4.21058421e+00, 4.73689474e+00, 5.26320526e+00, 5.78951579e+00,
       6.31582632e+00, 6.84213684e+00, 7.36844737e+00, 7.89475789e+00,
       8.42106842e+00, 8.94737895e+00, 9.47368947e+00, 1.00000000e+01]),
                         'EL__l1_ratio': array([1.000e-04, 1.112e-01, 2.223e-01, 3.334e-01, 4.445e-01, 5.556e-01,
       6.667e-01, 7.778e-01, 8.889e-01, 1.000e+00])})

In [36]:
print("Best params : ",gridscv.best_params_)
print("Best score  : ",gridscv.best_score_)

Best params :  {'EL__alpha': 10.0, 'EL__l1_ratio': 1.0}
Best score  :  0.5084207009081492


- `.varbos()` To see the Exicution process of `GridSearchCV()`

In [38]:
params = {'EL__alpha': np.linspace(0.0001,10,20), 'EL__l1_ratio': np.linspace(0.0001,1,10)}    # 'alpha' & 'l1_ratio' are the parameters of elastic net algo
gridscv = GridSearchCV(pipe, param_grid=params,verbose=3)
gridscv.fit(X,y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV 1/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.441 total time=   0.0s
[CV 2/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.502 total time=   0.0s
[CV 3/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.463 total time=   0.0s
[CV 4/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.519 total time=   0.0s
[CV 5/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.617 total time=   0.0s
[CV 1/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.441 total time=   0.0s
[CV 2/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.502 total time=   0.0s
[CV 3/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.463 total time=   0.0s
[CV 4/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.519 total time=   0.0s
[CV 5/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.617 total time=   0.0s
[CV 1/5] END EL__alpha=0.0001, 

GridSearchCV(estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000023B2AF99160>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000023B2CAACDA0>)],
                                                          verbos...
       2.10534211e+00, 2.63165263e+00, 3.15796316e+00, 3.68427368e+00,
       4.21058421e+00, 4.73689474e+00, 5.26320526e+00, 5.78951579e+00,
       6.31582632e+00, 6.84213684e+00, 7.36844737e+00, 7.89475789e+00,
       8.42106842e+00, 8.94737895e+00, 9.47368947e+00, 1.00000000e+01]),
                         'EL__l1_ratio': array([1.000e-04, 1.112e-01, 2.223e-01, 3.334e-01, 4.445e-01, 5.556e-01,
       6.667e-01, 7.778e-01, 8.889e-01, 1.000e+00])},
             verbose=3)